<a href="https://colab.research.google.com/github/yamadashamoji/tools/blob/main/006_date_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# @title 日付計算アプリ

!pip install -q python-dateutil
!pip install -q jpholiday

import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
from dateutil.relativedelta import relativedelta
import jpholiday
import calendar
from IPython.display import HTML

# --------------------------
# 共通関数：曜日名取得
# --------------------------
def get_weekday_name(date_obj):
    return ["月", "火", "水", "木", "金", "土", "日"][date_obj.weekday()]

# --------------------------
# ◆ 機能①：日付＋日数 → 計算
# --------------------------
base_date_picker = widgets.DatePicker(description='基準日:', value=datetime.date.today())
days_input = widgets.IntText(description='日数:', value=0)
calculate_button = widgets.Button(description='計算する', button_style='primary')
result_output = widgets.Output()

def calculate_date(b):
    with result_output:
        clear_output()
        base_date = base_date_picker.value
        days = days_input.value
        if base_date is None:
            print("⚠️ 基準日を選択してください。")
            return
        result_date = base_date + datetime.timedelta(days=days)
        weekday = get_weekday_name(result_date)
        print(f"📅 計算結果: {result_date}（{weekday}）")

calculate_button.on_click(calculate_date)
date_plus_ui = widgets.VBox([base_date_picker, days_input, calculate_button, result_output])

# --------------------------
# ◆ 機能②：2つの日付 → 差分
# --------------------------
start_date_picker = widgets.DatePicker(description='開始日:', value=datetime.date.today())
end_date_picker = widgets.DatePicker(description='終了日:', value=datetime.date.today())
diff_button = widgets.Button(description='差分を計算', button_style='success')
diff_output = widgets.Output()

def calculate_diff(b):
    with diff_output:
        clear_output()
        start = start_date_picker.value
        end = end_date_picker.value
        if start is None or end is None:
            print("⚠️ 開始日と終了日を選択してください。")
            return
        delta = (end - start).days
        print(f"🧮 差分: {abs(delta)}日（{min(start, end)} → {max(start, end)}）")

diff_button.on_click(calculate_diff)
date_diff_ui = widgets.VBox([start_date_picker, end_date_picker, diff_button, diff_output])

# --------------------------
# ◆ 機能③：月単位の加減
# --------------------------
month_base_picker = widgets.DatePicker(description='基準日:', value=datetime.date.today())
month_input = widgets.IntText(description='月数:', value=0)
month_button = widgets.Button(description='計算する', button_style='info')
month_output = widgets.Output()

def calculate_month_add(b):
    with month_output:
        clear_output()
        base = month_base_picker.value
        months = month_input.value
        if base is None:
            print("⚠️ 基準日を選択してください。")
            return
        result = base + relativedelta(months=months)
        weekday = get_weekday_name(result)
        print(f"📆 計算結果: {result}（{weekday}）")

month_button.on_click(calculate_month_add)
month_ui = widgets.VBox([month_base_picker, month_input, month_button, month_output])

# --------------------------
# ◆ 機能④：平日＋祝日除外の差分
# --------------------------
holiday_start_picker = widgets.DatePicker(description='開始日:', value=datetime.date.today())
holiday_end_picker = widgets.DatePicker(description='終了日:', value=datetime.date.today())
holiday_button = widgets.Button(description='平日のみ計算', button_style='warning')
holiday_output = widgets.Output()

def count_weekdays_exclude_holidays(b):
    with holiday_output:
        clear_output()
        start = holiday_start_picker.value
        end = holiday_end_picker.value
        if start is None or end is None:
            print("⚠️ 両方の日付を選んでください。")
            return

        day1 = min(start, end)
        day2 = max(start, end)
        count = 0
        for i in range((day2 - day1).days + 1):
            current = day1 + datetime.timedelta(days=i)
            if current.weekday() < 5 and not jpholiday.is_holiday(current):  # 平日かつ祝日でない
                count += 1
        print(f"🏢 平日（祝日除く）: {count}日（{day1}〜{day2}）")

holiday_button.on_click(count_weekdays_exclude_holidays)
holiday_ui = widgets.VBox([holiday_start_picker, holiday_end_picker, holiday_button, holiday_output])

# --------------------------
# ◆ 機能⑤：カレンダー表示（祝日付き）
# --------------------------

# 年・月指定用ウィジェット
calendar_year = widgets.IntText(description='年:', value=datetime.date.today().year)
calendar_month = widgets.IntSlider(description='月:', value=datetime.date.today().month, min=1, max=12)
calendar_button = widgets.Button(description='カレンダー表示', button_style='primary')
calendar_output = widgets.Output()

def display_calendar(b):
    with calendar_output:
        clear_output()
        year = calendar_year.value
        month = calendar_month.value
        cal = calendar.Calendar(firstweekday=6)  # 日曜始まり

        html = f"<h4>{year}年 {month}月のカレンダー</h4>"
        html += "<table style='border-collapse: collapse;'>"
        html += "<tr>" + "".join([f"<th style='padding:4px; border: 1px solid #999;'>{day}</th>" for day in ["日", "月", "火", "水", "木", "金", "土"]]) + "</tr>"

        today = datetime.date.today()
        for week in cal.monthdatescalendar(year, month):
            html += "<tr>"
            for day in week:
                is_today = (day == today)
                is_other_month = (day.month != month)
                is_holiday = jpholiday.is_holiday(day)

                style = "padding:4px; border: 1px solid #999; text-align: center;"
                if is_other_month:
                    style += "color: #aaa;"
                elif is_today:
                    style += "background-color: #ffeb3b;"  # yellow
                elif is_holiday or day.weekday() >= 5:
                    style += "color: red;"

                tooltip = ""
                if is_holiday:
                    name = jpholiday.is_holiday_name(day)
                    tooltip = f"title='{name}'"

                html += f"<td style='{style}' {tooltip}>{day.day}</td>"
            html += "</tr>"
        html += "</table>"

        display(HTML(html))

calendar_button.on_click(display_calendar)
calendar_ui = widgets.VBox([calendar_year, calendar_month, calendar_button, calendar_output])

# --------------------------
# ◆ UI 全体：タブ表示
# --------------------------
tabs = widgets.Tab(children=[
    date_plus_ui,
    date_diff_ui,
    month_ui,
    holiday_ui,
    calendar_ui
])

tabs.set_title(0, '日数で加算')
tabs.set_title(1, '日付の差分')
tabs.set_title(2, '月単位の加減')
tabs.set_title(3, '平日カウント')
tabs.set_title(4, 'カレンダー表示')

display(tabs)


In [7]:
# @title カレンダー表示（複数月表示用）

import calendar
import datetime
import jpholiday
from IPython.display import HTML, display
import ipywidgets as widgets

# ▼複数月表示ウィジェット
multi_year = widgets.IntText(description='年:', value=datetime.date.today().year)
multi_month = widgets.IntSlider(description='開始月:', value=datetime.date.today().month, min=1, max=12)
multi_count = widgets.IntSlider(description='表示月数:', value=3, min=1, max=12)
multi_button = widgets.Button(description='表示する', button_style='primary')
multi_output = widgets.Output()

# ▼カレンダー生成関数
def generate_month_calendar(year, month):
    cal = calendar.Calendar(firstweekday=6)
    html = f"<h4>{year}年 {month}月</h4>"
    html += "<table style='border-collapse: collapse; margin-bottom: 20px;'>"
    html += "<tr>" + "".join([f"<th style='padding:4px; border: 1px solid #999;'>{day}</th>" for day in ["日", "月", "火", "水", "木", "金", "土"]]) + "</tr>"

    today = datetime.date.today()
    for week in cal.monthdatescalendar(year, month):
        html += "<tr>"
        for day in week:
            is_today = (day == today)
            is_other_month = (day.month != month)
            is_holiday = jpholiday.is_holiday(day)

            style = "padding:4px; border: 1px solid #999; text-align: center;"
            if is_other_month:
                style += "color: #aaa;"
            elif is_today:
                style += "background-color: #ffeb3b;"
            elif is_holiday or day.weekday() >= 5:
                style += "color: red;"

            tooltip = f"title='{jpholiday.is_holiday_name(day)}'" if is_holiday else ""
            html += f"<td style='{style}' {tooltip}>{day.day}</td>"
        html += "</tr>"
    html += "</table>"
    return html

# ▼表示処理
def show_multiple_months(b):
    with multi_output:
        multi_output.clear_output()
        year = multi_year.value
        start_month = multi_month.value
        count = multi_count.value

        html_all = ""
        for i in range(count):
            y = year + (start_month + i - 1) // 12
            m = (start_month + i - 1) % 12 + 1
            html_all += generate_month_calendar(y, m)

        display(HTML(html_all))

multi_button.on_click(show_multiple_months)
multi_calendar_ui = widgets.VBox([
    widgets.HTML("<h3>📅 複数月カレンダー表示</h3>"),
    multi_year,
    multi_month,
    multi_count,
    multi_button,
    multi_output
])
display(multi_calendar_ui)

In [14]:
!apt-get install -y libpangocairo-1.0-0
!pip install -q weasyprint cairosvg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libpangocairo-1.0-0 is already the newest version (1.50.6+ds-2ubuntu1).
libpangocairo-1.0-0 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00


In [16]:
!apt-get install -y poppler-utils
!pip install -q weasyprint pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.7 [186 kB]
Fetched 186 kB in 0s (759 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126765 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.7_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.7) ...
Setting up poppler-utils (22.02.0-2ubuntu0.7) ...
Processing triggers for man-db (2.10.2-1) ...


In [17]:
from weasyprint import HTML as WeasyHTML
from weasyprint.text.fonts import FontConfiguration
from pdf2image import convert_from_path
from IPython.display import display, HTML
import os
import datetime
import calendar
import jpholiday
import ipywidgets as widgets
from google.colab import files

# ▼HTML生成（複数月分）
def generate_full_calendar_html(year, start_month, count):
    cal = calendar.Calendar(firstweekday=6)
    today = datetime.date.today()

    html = "<style>td, th { padding:4px; border:1px solid #999; text-align:center; }</style>"
    for i in range(count):
        y = year + (start_month + i - 1) // 12
        m = (start_month + i - 1) % 12 + 1
        html += f"<h4>{y}年 {m}月</h4>"
        html += "<table><tr>" + "".join([f"<th>{d}</th>" for d in ["日", "月", "火", "水", "木", "金", "土"]]) + "</tr>"

        for week in cal.monthdatescalendar(y, m):
            html += "<tr>"
            for day in week:
                is_today = (day == today)
                is_other_month = (day.month != m)
                is_holiday = jpholiday.is_holiday(day)

                style = ""
                if is_other_month:
                    style += "color: #aaa;"
                elif is_today:
                    style += "background-color: #ffeb3b;"
                elif is_holiday or day.weekday() >= 5:
                    style += "color: red;"

                tooltip = f"title='{jpholiday.is_holiday_name(day)}'" if is_holiday else ""
                html += f"<td style='{style}' {tooltip}>{day.day}</td>"
            html += "</tr>"
        html += "</table><br>"

    return html

# ▼保存処理
def save_calendar_as_image_pdf(year, start_month, count):
    html = generate_full_calendar_html(year, start_month, count)
    filename_base = f"calendar_{year}_{start_month}_{count}months"

    os.makedirs("calendar_output", exist_ok=True)
    html_path = f"calendar_output/{filename_base}.html"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html)

    # PDF出力
    pdf_path = f"calendar_output/{filename_base}.pdf"
    font_config = FontConfiguration()
    WeasyHTML(string=html).write_pdf(pdf_path, font_config=font_config)

    # PNG出力（PDFから画像へ）
    png_path = f"calendar_output/{filename_base}.png"
    images = convert_from_path(pdf_path)
    images[0].save(png_path, 'PNG')  # 最初のページのみ保存（カレンダーは1ページのはず）

    print("✅ PDFとPNGを保存しました！")
    files.download(pdf_path)
    files.download(png_path)

# ▼UIウィジェット
year_input = widgets.IntText(description="年:", value=datetime.date.today().year)
month_input = widgets.IntSlider(description="開始月:", value=datetime.date.today().month, min=1, max=12)
count_input = widgets.IntSlider(description="表示月数:", value=3, min=1, max=12)
save_button = widgets.Button(description="PNG & PDF 保存", button_style='success')
save_output = widgets.Output()

def on_save_clicked(b):
    with save_output:
        save_output.clear_output()
        save_calendar_as_image_pdf(year_input.value, month_input.value, count_input.value)

save_button.on_click(on_save_clicked)

# ▼表示UI
calendar_save_ui = widgets.VBox([
    widgets.HTML("<h3>🖨️ カレンダーをPNG & PDFで保存</h3>"),
    year_input,
    month_input,
    count_input,
    save_button,
    save_output
])
display(calendar_save_ui)

---

## 📄 日付計算アプリ 仕様書（Google Colab対応・Python）

### 🎯 アプリ概要
ユーザーが基準日と日数を入力すると、「何日後」「何日前」の日付を計算して表示するアプリケーション。

---

### 🛠️ 開発環境
- **使用言語**：Python 3.x  
- **実行環境**：Google Colaboratory  
- **ライブラリ**：
  - `datetime`（標準ライブラリ）
  - `IPython.display`（Colab用UI補助）
  - `ipywidgets`（インタラクティブUI）

---

### 📌 主な機能

| 機能名 | 内容 |
|--------|------|
| 基準日入力 | ユーザーが計算の基準となる日付をカレンダーまたはテキストで入力できる |
| 日数入力 | 「何日後」「何日前」などの加減日数を整数で入力可能（正負対応） |
| 結果表示 | 計算された日付を見やすく表示 |
| 日付フォーマット対応 | 結果を`YYYY-MM-DD`形式で表示。必要に応じて和暦や曜日なども表示可能 |
| 双方向計算 | 「未来の日付 → 何日後か」「過去の日付 → 何日前か」など逆算も可能（※オプション） |
| UI対応 | `ipywidgets`によるGUIで操作（カレンダー・スライダー・ボタンなど） |

---

### 🧩 オプション機能（将来的に追加可能）

- 曜日表示（例：2025-04-13（日））
- 範囲計算（指定した2つの日付間の日数を計算）
- 月単位・年単位の加減（日付 + 月、日付 + 年）
- 祝日考慮（日本の祝日を除外して計算）

---

### 🖼️ 画面イメージ（UI構成）

```
📅 基準日：[ 2025-04-13 ]（カレンダーウィジェット）
🔢 日数：[ -5 ] 日（スライダー or テキスト入力）
📌 結果：2025-04-08（火）

[ 計算する ] ボタン
```

---

### ✅ 処理フロー（基本的なロジック）

1. 基準日と加減する日数を取得
2. `datetime.timedelta`を使用して日付計算
3. 結果を所定の形式で表示
4. 入力が不正な場合はエラーメッセージ表示

---